https://qiita.com/taka_yayoi/items/a973fa2d08062224d422

# loading dataset

In [7]:
from datasets import load_dataset
from tqdm.notebook import tqdm

dataset_name = "taka-yayoi/databricks-dolly-15k-ja"
dataset = load_dataset(dataset_name, split="train")

Found cached dataset json (/home/jupyter/.cache/huggingface/datasets/taka-yayoi___json/taka-yayoi--databricks-dolly-15k-ja-2f260017aa28eec2/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


# prompt template

In [8]:
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    input_key=INPUT_KEY,
    input="{input}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY
)

def apply_prompt_template(examples):
    instruction = examples["instruction"]
    response = examples["response"]
    context = examples.get("context")

    if context:
        full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
    else:
        full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
    return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/taka-yayoi___json/taka-yayoi--databricks-dolly-15k-ja-2f260017aa28eec2/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-0b935ced85d8d0a7.arrow


In [9]:
dataset["text"][0]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nヴァージン・オーストラリア航空はいつから運航を開始したのですか？\n\nInput:\nヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・グループを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリア航空の破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。\n\n### Response:\nヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。\n\n### End'

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "elinas/llama-7b-hf-transformers-4.29"
revision = "d33594ee64ef1b6264543b6a88f60982a55fdb7a"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="auto",
    revision=revision,
    trust_remote_code=True
)
model.config.use_cache = False


RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
[Errno 13] Permission denied: '/root/google_vm_config.lock'